In [ ]:
import sys
sys.path.append("..")
sys.path.append("my_library")
from my_library.config import *
from my_library.metrics.cmp_metrics import *
sys.path.append("..")
import os

In [ ]:
import pandas as pd
csv_df = pd.read_csv(working_directory+"/projects/I3lung-sqadqc-project/annotations.csv")


# setup check

In [ ]:
csv_df

In [ ]:
#utcome = "PDL1_CATHEGORY"
#model = "pdl1-3"

# Find all the paths

In [ ]:
import os

root_dir = f"{working_directory}/{eval}/train/" + model

train_parquet_paths = []
val_parquet_paths = []
test_parquet_paths = []

for dirpath, dirnames, filenames in os.walk(root_dir):
    # Check if the folder name contains "attention"
    if "attention" in os.path.basename(dirpath).lower():

        # If you want to do something with each file inside those folders
        for f in filenames:
            file_path = os.path.join(dirpath, f)
            if f.lower().endswith(".parquet"):
                #print(f"   This is a Parquet file: {file_path}")
                train_parquet_paths.append(file_path)
train_parquet_paths.sort()

#print("Sorted Parquet paths:")
#for path in train_parquet_paths:
#    print(path)

root_dir = f"{working_directory}/{eval}/val/" + model

for dirpath, dirnames, filenames in os.walk(root_dir):
    # Check if the folder name contains "attention"
    if "attention" in os.path.basename(dirpath).lower():

        # If you want to do something with each file inside those folders
        for f in filenames:
            file_path = os.path.join(dirpath, f)
            if f.lower().endswith(".parquet"):
                #print(f"   This is a Parquet file: {file_path}")
                val_parquet_paths.append(file_path)
val_parquet_paths.sort()


root_dir = f"{working_directory}/{eval}/test-int/" + model

for dirpath, dirnames, filenames in os.walk(root_dir):
    # Check if the folder name contains "attention"
    if "attention" in os.path.basename(dirpath).lower():

        # If you want to do something with each file inside those folders
        for f in filenames:
            file_path = os.path.join(dirpath, f)
            if f.lower().endswith(".parquet"):
                #print(f"   This is a Parquet file: {file_path}")
                test_parquet_paths.append(file_path)
test_parquet_paths.sort()


In [ ]:
val_parquet_paths

In [ ]:
test_parquet_paths

# Train Dummy models

In [ ]:
import os
import torch
import numpy as np
import pandas as pd
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.metrics import classification_report, accuracy_score

In [ ]:
metrics_list = []
numbers = []
desired_order = ["< 1 %", "1-49 %", ">=50 %"]

train_models = []
train_classification_reports = []

train_true_y=[]
train_pred_y =[]

for i in train_parquet_paths:
    df_predictions = pd.read_parquet(i)
    df_joined = pd.merge(df_predictions, csv_df, on ='slide', how = 'left')
    #print(df_joined)
    distribution = df_joined[["slide", outcome]]

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    x_list = np.ones((distribution.shape[0], 1))
    y_list = df_joined[outcome].to_numpy()

    dummy_clf = DummyClassifier(strategy="most_frequent")
    dummy_clf.fit(x_list, y_list)

    y_pred = dummy_clf.predict(x_list)

    train_models.append(dummy_clf)
    
   
    train_true_y.append(y_list)
    train_pred_y.append(y_pred)


In [ ]:
len(train_true_y)

In [ ]:
'''import numpy as np

# Assuming your list of dictionaries is stored in the variable `reports`
reports = train_classification_reports  # your list of dictionaries

def average_report_group(group):
    """Average a group of report dictionaries."""
    avg_report = {}
    # Loop over each key in the first dictionary of the group.
    for key in group[0]:
        if key == 'accuracy':
            # Average the accuracy values directly.
            avg_report[key] = np.mean([r[key] for r in group])
        else:
            # For keys that are nested dictionaries, average each inner metric.
            inner_keys = group[0][key].keys()
            avg_inner = {}
            for metric in inner_keys:
                avg_inner[metric] = np.mean([r[key][metric] for r in group])
            avg_report[key] = avg_inner
    return avg_report

group_size = 5
averaged_reports = []

# Process the reports list in chunks of 5.
for i in range(0, len(reports), group_size):
    group = reports[i:i + group_size]
    avg = average_report_group(group)
    averaged_reports.append(avg)

# Now `averaged_reports` contains the averaged dictionary for each group of 5 items.
print(averaged_reports)

df_averaged_pred = pd.DataFrame(averaged_reports)'''


In [ ]:
metrics_list = []
numbers = []
desired_order = ["< 1 %", "1-49 %", ">=50 %"]

val_models = []
val_classification_reports = []

val_true_y = []
val_pred_y = []

for i in val_parquet_paths:
    df_predictions = pd.read_parquet(i)
    df_joined = pd.merge(df_predictions, csv_df, on ='slide', how = 'left')
    #print(df_joined)
    distribution = df_joined[["slide", outcome]]

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    x_list = np.ones((distribution.shape[0], 1))
    y_list = df_joined[outcome].to_numpy()

    dummy_clf = DummyClassifier(strategy="most_frequent")
    dummy_clf.fit(x_list, y_list)

    y_pred = dummy_clf.predict(x_list)

    val_models.append(dummy_clf)

    val_true_y.append(y_list)
    val_pred_y.append(y_pred)

In [ ]:
print(test_parquet_paths)

In [ ]:
metrics_list = []
numbers = []
desired_order = ["< 1 %", "1-49 %", ">=50 %"]

test_scores = []
test_true_y = []
test_pred_y =[]

test_list_dataframes = []

for i, path in enumerate(test_parquet_paths):
    df_predictions = pd.read_parquet(path)
    dummy_clf = train_models[i]
    df_joined = pd.merge(df_predictions, csv_df, on ='slide', how = 'left')
    #print(df_joined)
    distribution = df_joined[["slide",outcome]]

    display(distribution)

    y_true = df_joined[outcome].to_numpy()
    print(y_true)
    x_list = np.ones((distribution.shape[0], 1))



    #print(distribution)
    y_pred = dummy_clf.predict(x_list)
    print(y_pred)

    test_true_y.append(y_true)
    test_pred_y.append(y_pred)

    distribution["y_pred"] = y_pred
    test_list_dataframes.append(distribution)




In [ ]:
print(len(test_list_dataframes))

In [ ]:
test_df = pd.concat(test_list_dataframes, ignore_index = True)

In [ ]:
distribution

In [ ]:
y_true

In [ ]:
'''import numpy as np
import pandas as pd

# Your list of dictionaries (only a few items are shown for brevity; include all in your actual code)
report = train_classification_reports

def average_report_group_with_print(group, group_index):
    print(f"\nProcessing group {group_index} (with {len(group)} items):")
    group_stats = {}
    for key in group[0]:
        if key == 'accuracy':
            values = [r[key] for r in group]
            avg_val = np.mean(values)
            print(f"  {key}: values = {values}, average = {avg_val:.4f}")
            group_stats[f"{key}_avg"] = avg_val
        else:
            # For nested dictionary keys (like '1-49 %', '< 1 %', etc.)
            for subkey in group[0][key]:
                values = [r[key][subkey] for r in group]
                avg_val = np.mean(values)
                print(f"  {key} -> {subkey}: values = {values}, average = {avg_val:.4f}")
                flat_key = f"{key}_{subkey}"
                group_stats[f"{flat_key}_avg"] = avg_val
    return group_stats

group_size = 5
averaged_stats = []

for i in range(0, len(reports), group_size):
    group = reports[i:i+group_size]
    group_index = i // group_size
    group_stats = average_report_group_with_print(group, group_index)
    averaged_stats.append(group_stats)

# Create a DataFrame from the averaged statistics.
df_train = pd.DataFrame(averaged_stats)

print("\nFinal DataFrame with averaged values:")
print(df_train)'''

In [ ]:
'''df_train=df_train.reset_index()
df_val=df_val.reset_index()
df_test=df_test.reset_index()
''''''
df_train.columns = ["Total number of patients", "< 1 %", "1-49 %", ">=50 %"]
df_val .columns = ["Total number of patients", "< 1 %", "1-49 %", ">=50 %"]
df_test.columns = ["Total number of patients", "< 1 %", "1-49 %", ">=50 %"]'''
'''


df_train_prefixed = df_train.add_suffix("_train")
df_val_prefixed   = df_val.add_suffix("_val")
df_test_prefixed  = df_test.add_suffix("_test")

df_final = pd.concat([df_train_prefixed, df_val_prefixed, df_test_prefixed], axis=1)
print(df_final)'''


In [ ]:
'''import numpy as np
import pandas as pd

# Your list of dictionaries (using your provided sample; in your code include all items)
reports = train_classification_reports

# Number of items
n = len(reports)
print(f"Total number of reports: {n}\n")

# Function to compute average and CI (CI = 2*std/sqrt(n))
def compute_avg_and_ci(values):
    avg = np.mean(values)
    std = np.std(values, ddof=1) if len(values) > 1 else 0.0
    ci = 2 * std / np.sqrt(len(values)) if len(values) > 1 else 0.0
    return avg, ci

# Dictionary to hold our flattened average and CI values
overall_stats = {}

# Process each key in the first report as a template.
for key in reports[0]:
    if key == 'accuracy':
        # Collect the accuracy values
        values = [r[key] for r in reports]
        avg, ci = compute_avg_and_ci(values)
        print(f"Metric '{key}':")
        print(f"  Values: {values}")
        print(f"  Average = {avg:.4f}, CI = {ci:.4f}\n")
        overall_stats[f"{key}_avg"] = avg
        overall_stats[f"{key}_ci"] = ci
    else:
        # For nested dictionary keys (like '1-49 %', '< 1 %', etc.)
        for subkey in reports[0][key]:
            values = [r[key][subkey] for r in reports]
            avg, ci = compute_avg_and_ci(values)
            print(f"Metric '{key}' -> '{subkey}':")
            print(f"  Values: {values}")
            print(f"  Average = {avg:.4f}, CI = {ci:.4f}\n")
            flat_key = f"{key}_{subkey}"
            overall_stats[f"{flat_key}_avg"] = avg
            overall_stats[f"{flat_key}_ci"] = ci

# Create a DataFrame from the overall statistics dictionary.
df = pd.DataFrame([overall_stats])
print("Final DataFrame with overall averages and CIs:")
display(df)
df.to_csv("pdl1-dummy-model-train.csv", index=True)'''


In [ ]:
'''import numpy as np
import pandas as pd

# Your list of dictionaries (using your provided sample; in your code include all items)
reports = val_classification_reports

# Number of items
n = len(reports)
print(f"Total number of reports: {n}\n")

# Function to compute average and CI (CI = 2*std/sqrt(n))
def compute_avg_and_ci(values):
    avg = np.mean(values)
    std = np.std(values, ddof=1) if len(values) > 1 else 0.0
    ci = 2 * std / np.sqrt(len(values)) if len(values) > 1 else 0.0
    return avg, ci

# Dictionary to hold our flattened average and CI values
overall_stats = {}

# Process each key in the first report as a template.
for key in reports[0]:
    if key == 'accuracy':
        # Collect the accuracy values
        values = [r[key] for r in reports]
        avg, ci = compute_avg_and_ci(values)
        print(f"Metric '{key}':")
        print(f"  Values: {values}")
        print(f"  Average = {avg:.4f}, CI = {ci:.4f}\n")
        overall_stats[f"{key}_avg"] = avg
        overall_stats[f"{key}_ci"] = ci
    else:
        # For nested dictionary keys (like '1-49 %', '< 1 %', etc.)
        for subkey in reports[0][key]:
            values = [r[key][subkey] for r in reports]
            avg, ci = compute_avg_and_ci(values)
            print(f"Metric '{key}' -> '{subkey}':")
            print(f"  Values: {values}")
            print(f"  Average = {avg:.4f}, CI = {ci:.4f}\n")
            flat_key = f"{key}_{subkey}"
            overall_stats[f"{flat_key}_avg"] = avg
            overall_stats[f"{flat_key}_ci"] = ci

# Create a DataFrame from the overall statistics dictionary.
df = pd.DataFrame([overall_stats])
print("Final DataFrame with overall averages and CIs:")
display(df)
df.to_csv("pdl1-dummy-model-val.csv", index=True)'''

In [ ]:
'''import numpy as np
import pandas as pd

# Your list of dictionaries (using your provided sample; in your code include all items)
reports = test_scores

# Number of items
n = len(reports)
print(f"Total number of reports: {n}\n")

# Function to compute average and CI (CI = 2*std/sqrt(n))
def compute_avg_and_ci(values):
    avg = np.mean(values)
    std = np.std(values, ddof=1) if len(values) > 1 else 0.0
    ci = 2 * std / np.sqrt(len(values)) if len(values) > 1 else 0.0
    return avg, ci

# Dictionary to hold our flattened average and CI values
overall_stats = {}

# Process each key in the first report as a template.
for key in reports[0]:
    if key == 'accuracy':
        # Collect the accuracy values
        values = [r[key] for r in reports]
        avg, ci = compute_avg_and_ci(values)
        print(f"Metric '{key}':")
        print(f"  Values: {values}")
        print(f"  Average = {avg:.4f}, CI = {ci:.4f}\n")
        overall_stats[f"{key}_avg"] = avg
        overall_stats[f"{key}_ci"] = ci
    else:
        # For nested dictionary keys (like '1-49 %', '< 1 %', etc.)
        for subkey in reports[0][key]:
            values = [r[key][subkey] for r in reports]
            avg, ci = compute_avg_and_ci(values)
            print(f"Metric '{key}' -> '{subkey}':")
            print(f"  Values: {values}")
            print(f"  Average = {avg:.4f}, CI = {ci:.4f}\n")
            flat_key = f"{key}_{subkey}"
            overall_stats[f"{flat_key}_avg"] = avg
            overall_stats[f"{flat_key}_ci"] = ci

# Create a DataFrame from the overall statistics dictionary.
df = pd.DataFrame([overall_stats])
print("Final DataFrame with overall averages and CIs:")
display(df)
df.to_csv("pdl1-dummy-model-test.csv", index=True)'''

In [ ]:
'''display(df_final)'''

In [ ]:
'''df_final.to_csv("pdl1-dummy-model.csv", index=True)'''

In [ ]:
def extract_5_elements (train_true_y, train_pred_y):
    liste =train_true_y
    y_trues = []
    for i in range(0, len(liste), 5):
        unione = [item for sublist in liste[i:i+5] for item in sublist]
        y_trues.append(unione)
        #print(unione)

    liste = train_pred_y
    y_preds = []
    for i in range(0, len(liste), 5):
        unione = [item for sublist in liste[i:i+5] for item in sublist]
        y_preds.append(unione)
        #print(unione)
    return y_trues, y_preds


In [ ]:
#y_trues, y_preds = extract_5_elements(train_true_y, train_pred_y)

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
def average(df_input, name): 
    df = df_input.copy()
    # Removing "Fold" column
    df = df.drop(columns=["Dataset","Fold"])  
    mean_values = df.mean()
    std_values = df.std()
    n = len(df)
    ci_values = (2 * std_values) / np.sqrt(n)
    result_df = pd.DataFrame({
    "Column": mean_values.index,
    "Mean": mean_values.values,
    "CI": ci_values.values
    })
    result_df = result_df.add_prefix(name)
    display(result_df)
    return result_df

In [ ]:
def extract_metrics(train_true_y, train_pred_y, model, name_set):
    y_trues, y_preds = extract_5_elements(train_true_y, train_pred_y)
    results = []  # Store results

    for i in range(len(y_trues)):
        y_list = y_trues[i]
        y_pred = y_preds[i]
        acc = accuracy_score(y_list, y_pred)
        precision = precision_score(y_list, y_pred, average="weighted", zero_division=0)
        recall = recall_score(y_list, y_pred, average="weighted", zero_division=0)
        f1 = f1_score(y_list, y_pred, average="weighted", zero_division=0)

        report = classification_report(y_list, y_pred, digits=2, output_dict=True, zero_division=0)

        # Flatten report into a list
        row = {"Dataset": name_set, "Fold": i+1, "Accuracy": acc, "Precision" : precision, "Recall": recall, "F1_score" : f1}

        for label, metrics in report.items():
            if isinstance(metrics, dict):  # Avoid 'accuracy' which is a single value
                for metric_name, value in metrics.items():
                    row[f"{label}_{metric_name}"] = value
        
        results.append(row)

    # Convert to DataFrame
    df_results = pd.DataFrame(results)


    display(df_results)
    #df_results.to_excel(model+f"_{name_set}_5_folds.xlsx")

    avg = average(df_results, name_set)

    return df_results, avg
    


In [ ]:
'''
for i in range (5):
    print(len(y_trues[i]))

for i in range (5):
    print(len(y_preds[i]))'''

In [ ]:
df1, avg_1 = extract_metrics( train_true_y, train_pred_y, model, "train")
df2, avg_2= extract_metrics( val_true_y, val_pred_y, model, "val")
df3, avg_3 = extract_metrics( test_true_y, test_pred_y, model, "test")

df_final = pd.concat([df1, df2, df3], axis=0, ignore_index=True)
#df_final.to_excel(model+f"_complete_5_folds.xlsx")

#df_final_avg = pd.concat([avg_1, avg_2, avg_3], axis=1, ignore_index=False)
#df_final_avg.to_excel(model+f"_complete_averages_5_folds.xlsx")

In [ ]:
df_final

In [ ]:
test_df =test_df.rename(columns ={outcome: "y_true"})

In [ ]:
test_df

In [ ]:
# Raggruppiamo i dati per slide e y_true e creiamo nuove colonne per ogni valore di y_pred_max_idx
df_final = test_df.groupby(["slide", "y_true"])["y_pred"].agg(list).reset_index()

# Espandiamo la lista in colonne separate
df_final_expanded = df_final["y_pred"].apply(pd.Series)

# Rinominiamo le nuove colonne
df_final_expanded.columns = [f"pred_{i+1}" for i in df_final_expanded.columns]

# Concatenazione con le colonne originali
df_result = pd.concat([df_final.drop(columns=["y_pred"]), df_final_expanded], axis=1)

In [ ]:
df_result

In [ ]:
a = [col for col in df_result.columns if 'pred_' in col]

In [ ]:

# Supponiamo che il tuo DataFrame si chiami df
colonne_pred = [col for col in df_result.columns if 'pred_' in col]#[f"pred_{i}" for i in range(1, 6)]

# Calcola il valore più frequente (majority vote) per ogni riga
df_result['y_pred'] = df_result[colonne_pred].mode(axis=1)[0]

# Visualizza il DataFrame aggiornato
print(df_result.head())

In [ ]:
df_test = df_result[["slide", "y_true", "y_pred"]]

In [ ]:
df_test

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report
def extract_metrics(train_true_y, train_pred_y, model, name_set):
    
    results = []  # Store results
    y_list = train_true_y
    y_pred = train_pred_y
    acc = accuracy_score(y_list, y_pred)

    precision = precision_score(y_list, y_pred, average="weighted", zero_division=0)
    weighted_accuracy = balanced_accuracy_score(y_list, y_pred)

    recall = recall_score(y_list, y_pred, average="weighted", zero_division=0)
    f1 = f1_score(y_list, y_pred, average="weighted", zero_division=0)

    report = classification_report(y_list, y_pred, digits=2,   output_dict=True, zero_division=0)

    # Flatten report into a list
    row = {"Dataset": name_set, "Fold": i+1, "Accuracy": acc, "Balanced Accuracy":weighted_accuracy, "Precision" : precision, "Recall": recall, "F1_score" : f1}

    for label, metrics in report.items():
        if isinstance(metrics, dict):  # Avoid 'accuracy' which is a single value
            for metric_name, value in metrics.items():
                row[f"{label}_{metric_name}"] = value
        
    results.append(row)

    # Convert to DataFrame
    df_results = pd.DataFrame(results)


    display(df_results)
    #df_results.to_excel(model+f"_{name_set}_5_folds.xlsx")

    

    return df_results

In [ ]:
y_true = df_test["y_true"].to_numpy()
y_pred = df_test["y_pred"].to_numpy()

In [ ]:
df = extract_metrics(y_true, y_pred, "br3", "test")